# Transmitting Informations

Nous avons vu qu'il était possible de passer des donées du `serveur flask` -> `client` en utilisant la méthode `render_template` de `flask`.

Peut-on faire l'inverse ? C'est à dire envoyer des informations du `client` au `serveur` ?

## Les formulaires

Pour envoyer des informations du `client` au `serveur`, on utilise des formulaires.

Un formulaire est un ensemble de champs de saisie (balise `input`) qui permettent à l'utilisateur de saisir des informations.

### Créer un formulaire

Pour créer un formulaire, on utilise la balise `form` qui contient les balises `input`.

```html
<form action="/some_route" method="POST">  
  <input type="text" name="prenom" placeholder="Votre prénom">
  <input type="text" name="nom" placeholder="Votre nom">
  <input type="submit" value="Envoyer">
</form>
```

Ici nous avons créé un formulaire avec deux champ de saisie pour le nom et le prénom et un bouton pour envoyer le formulaire (`type="submit"`).

### Les attributs `action` et `method`

La balise `<form>`, prend deux attributs (paramètres) :
- `action` : l'URL (endpoint) de la page qui va traiter les informations du formulaire.
- `method` : la méthode HTTP à utiliser pour envoyer les informations du formulaire`

> **/!\ en HTML seulement des methodes HTTP sont acceptés: `GET` et `POST`**

### Les champs de saisie

Les champs de saisie sont définis par la balise `input`. Cette balise prend plusieurs attributs :
- `type` : le type de champ de saisie (text, password, email, number, ...)
- `name` : le nom du champ de saisie
- `placeholder` : le texte qui s'affiche dans le champ de saisie
- `value` : la valeur par défaut du champ de saisie
- `required` : le champ de saisie est obligatoire
- `readonly` : le champ de saisie est en lecture seule
- `disabled` : le champ de saisie est désactivé

Ici, le name est très important car c'est grâce à lui que l'on va pouvoir récupérer les informations du formulaire côté serveur.

## Récupérer les informations du formulaire

Pour récupérer les informations du formulaire côté serveur, on doit définir une route (celle qui sera utilisée dans l'attribut `action` du formulaire)

On doit aussi spécifier que cette route accepte les méthodes `GET` et/ou `POST`

### Object `request`

Pour récupérer les informations du formulaire, on utilise l'objet `request` de `flask`.

Cet objet contient toutes les informations de la requête HTTP (méthode, headers, cookies, données, ...)

Pour récupérer les informations du formulaire, on utilise l'attribut `form` de l'objet `request`.

l'attribut `form` est un dictionnaire qui contient les informations du formulaire. Sous la forme `clé:valeur`, où la clé est le nom (`name`) du champ de saisie et la valeur est la valeur saisie par l'utilisateur.

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/form', methods=['GET', 'POST'])
def form():
  if request.method == 'POST': # request object has attribute method, which is a string informing the method used in the request
    name = request.form['name'] # request object has attribute form, which is a dictionary with the form data
    lastname = request.form['lastname'] # Here I'm getting the value of the form for the input with name attribute 'lastname'
    return 'You submitted the form: the name is %s and the lastname is %s' % (name, lastname)
  return '''
    <form method="post">
      <input type="text" name="name">
      <input type="text" name="lastname">
      <input type="submit">
    </form>
  '''

if __name__ == '__main__':
  app.run()

## Sending files

Il est possible via un formulaire d'envoyer des fichiers (images, vidéos, documents, ...)

Pour cela, on utilise la balise `input` avec l'attribut `type="file"` 
Il va falloir aussi ajouter l'attribut `enctype="multipart/form-data"` dans la balise `form`.

```html
<form action="" method="POST" enctype="multipart/form-data">
  <input type="file" name="fileInput">
  <input type="submit" value="Envoyer">
</form>
```

Pour récupérer le fichier côté serveur, on utilise l'attribut `files` de l'objet `request`.

L'attribut `files` est un dictionnaire qui contient les fichiers envoyés par le client. La clé est le nom du champ de saisie et la valeur est un objet `FileStorage` qui contient les informations du fichier.

infomration sur le fichier:
- `filename` : le nom du fichier
- `content_type` : le type de contenu du fichier
- `content_length` : la taille du fichier
- `read()` : lire le contenu du fichier
- `save()` : sauvegarder le fichier sur le serveur

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/form/files', methods=['GET', 'POST'])
def form_files():
  if request.method == 'POST': # request object has attribute method, which is a string informing the method used in the request
    file = request.files['fileInput']
    file.save(file.filename)
    return 'File uploaded successfully'
  return '''
    <form action="" method="POST" enctype="multipart/form-data">
      <input type="file" name="fileInput">
      <input type="submit" value="Envoyer">
    </form>
  '''

if __name__ == '__main__':
  app.run()

#### Multiple files

Il est possible d'envoyer plusieurs fichiers en même temps en ajoutant l'attribut `multiple` à la balise `input`.

```html
<form action="" method="POST" enctype="multipart/form-data">
  <input type="file" name="fileInput" multiple>
  <input type="submit" value="Envoyer">
</form>
```

Pour récupérer les fichiers côté serveur, on utilise l'attribut `getlist` de l'objet `files`.

```python
files = request.files.getlist('fileInput')
```

Ceci retourne une liste d'objets `FileStorage` qui contiennent les informations des fichiers.

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/form/files', methods=['GET', 'POST'])
def form_files():
  if request.method == 'POST': # request object has attribute method, which is a string informing the method used in the request
    files = request.files.getlist('fileInput')
    for file in files:
      file.save(file.filename)
    return 'File uploaded successfully'
  return '''
    <form action="" method="POST" enctype="multipart/form-data">
      <input type="file" name="fileInput">
      <input type="submit" value="Envoyer">
    </form>
  '''

if __name__ == '__main__':
  app.run()

#### File types filtering

Il est possible de filtrer les types de fichiers acceptés en ajoutant l'attribut `accept` à la balise `input`.

```html
<form action="" method="POST" enctype="multipart/form-data">
  <input type="file" name="fileInput" accept="image/*">
  <input type="submit" value="Envoyer">
</form>
```

Ici, on accepte uniquement les fichiers de type image.

#### File Types

Il est possible de spécifier les types de fichiers acceptés en utilisant les types MIME (Multipurpose Internet Mail Extensions).

[Quelques examples](https://pro.europeana.eu/page/media-formats-mime-types)

#### File size filtering

Il est possible de filtrer la taille des fichiers acceptés en ajoutant l'attribut `size` à la balise `input`.

La taille est en octets (bytes).

```html
<form action="" method="POST" enctype="multipart/form-data">
  <input type="file" name="fileInput" size="1000000">
  <input type="submit" value="Envoyer">
</form>
```

Ici, on accepte uniquement les fichiers de taille inférieure à 1 Mo.

# WTF Forms

Pour simplifier la création de formulaires, `flask` propose une extension appelée `WTForms`.

`WTForms` permet de créer des formulaires en utilisant des classes Python.

## Fonctionnement

Pour créer un formulaire avec `WTForms`, on doit créer une classe qui hérite de `flask_wtf.FlaskForm`.

Dans cette classe, on définit les champs de saisie en utilisant des attributs de classe.

```python
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField

class MyForm(FlaskForm):
    name = StringField('Nom')
    lastname = StringField('Prénom')
    submit = SubmitField('Envoyer')
```

Ici, on a créé un formulaire avec deux champs de saisie pour le nom et le prénom et un bouton pour envoyer le formulaire.

## Les champs de saisie

Les champs de saisie sont définis par des classes qui héritent de `wtforms.fields`.

Il existe plusieurs types de champs de saisie :
- `StringField` : champ de saisie pour du texte
- `PasswordField` : champ de saisie pour un mot de passe
- `TextAreaField` : champ de saisie pour du texte multiligne
- `IntegerField` : champ de saisie pour un entier
- `DateField` : champ de saisie pour une date
- `SubmitField` : bouton pour envoyer le formulaire
- `FileField` : champ de saisie pour un fichier
- `SelectField` : champ de saisie pour une liste déroulante

## Les attributs des champs de saisie

Les champs de saisie prennent plusieurs arguments :
- `label` : le texte qui s'affiche à côté du champ de saisie
- `validators` : les validateurs à appliquer sur le champ de saisie
- `default` : la valeur par défaut du champ de saisie
- `render_kw` : les attributs HTML à ajouter à la balise `input`
- `choices` : les choix possibles pour le champ de saisie (pour `SelectField`)

## Les validateurs

Les validateurs permettent de valider les données saisies par l'utilisateur.

Il existe plusieurs validateurs :
- `DataRequired` : le champ de saisie est obligatoire
- `Length` : la longueur du champ de saisie
- `Email` : le champ de saisie doit être une adresse email
- `NumberRange` : le champ de saisie doit être un nombre dans une plage donnée
- `Regexp` : le champ de saisie doit correspondre à une expression régulière
- `EqualTo` : le champ de saisie doit être égal à un autre champ de saisie
- `FileAllowed` : le fichier doit avoir une extension donnée
- `FileRequired` : le champ de saisie est obligatoire
- `AnyOf` : le champ de saisie doit être dans une liste de valeurs

## Créer un formulaire

Pour créer un formulaire avec `WTForms`, on doit instancier la classe du formulaire dans la route.

In [ ]:
from wtforms import Form, EmailField, PasswordField, StringField
from wtforms.validators import email, DataRequired, equal_to

class RegisterForm(FlaskForm):
  email = EmailField('Email address', validators=[email(), DataRequired()])
  pwd = PasswordField('Password', validators=[DataRequired()])
  pwd_confirm = PasswordField('Confirm password', validators=[equal_to(pwd)])

> **/!\ pour utiliser le filtre email, il faut l'installer**
> ```bash
> pip install email-validator
> ```

## Afficher un formulaire | Python

Pour afficher un formulaire dans un template, on doit passer le formulaire en paramètre de la fonction `render_template`.

In [ ]:
@app.route('/register', methods=['GET', 'POST'])
def register():
  form = RegisterForm()
  if form.validate():
    email = form.email.data
    pwd = form.pwd.data
    pwd_confirm = form.pwd_confirm.data
    return f'Email: {email}, Password: {pwd}, Confirm password: {pwd_confirm}'
  return render_template('register.html', form=form)

Ici, on crée une instance du formulaire `RegisterForm` que l'on stocke dans la variable `form`.

La fonction `validate_on_submit` permet de valider les données du formulaire (basé sur les différents `validators`). 

Si les données sont valides, on récupère les données du formulaire avec l'attribut `data` de chaque champ de saisie. 
Cette fois on a un object `RegisterFor` qui as des attributs, qui sont les différents champs de saisie. 
Ce sont des objets aussi, et ils ont un attribut `data` qui contient la valeur saisie par l'utilisateur.

Finalemnt on passe le formulaire dans le template `register.html` pour l'afficher (`form=form`).

## Afficher un formulaire | HTML

```html
<form action="" method="POST">
  {{ form.email.label }} {{ form.email }}
  {{ form.pwd.label }} {{ form.pwd }}
  {{ form.pwd_confirm.label }} {{ form.pwd_confirm }}
  {{ form.submit }}
</form>
```

Ici, on affiche les champs de saisie du formulaire en utilisant les attributs de la classe `RegisterForm`.

## Les erreurs de validation

Si les données du formulaire ne sont pas valides, on peut afficher les erreurs de validation en utilisant l'attribut `errors` de chaque champ de saisie.

```html
<form action="" method="POST">
  {{ form.email.label }} {{ form.email }}
  {% for error in form.email.errors %}
    <p>{{ error }}</p>
  {% endfor %}
  {{ form.pwd.label }} {{ form.pwd }}
  {% for error in form.pwd.errors %}
    <p>{{ error }}</p>
  {% endfor %}
  {{ form.pwd_confirm.label }} {{ form.pwd_confirm }}
  {% for error in form.pwd_confirm.errors %}
    <p>{{ error }}</p>
  {% endfor %}
  {{ form.submit }}
</form>
```

Ici, on affiche les erreurs de validation pour chaque champ de saisie.

## Les messages flash

Pour afficher des messages flash, on utilise l'objet `flash` de `flask`.

In [ ]:
from flask import flash

@app.route('/register', methods=['GET', 'POST'])
def register():
  form = RegisterForm()
  if form.validate_on_submit():
    email = form.email.data
    pwd = form.pwd.data
    pwd_confirm = form.pwd_confirm.data
    flash('Inscription réussie', 'success')
    return f'Email: {email}, Password: {pwd}, Confirm password: {pwd_confirm}'
  return render_template('register.html', form=form)

Ici, on affiche un message flash avec le texte `Inscription réussie` et le type `success`.

Pour afficher les messages flash dans le template, on utilise la fonction `get_flashed_messages`.

```html
{% with messages = get_flashed_messages() %}
  {% if messages %}
    <ul>
      {% for message in messages %}
        <li>{{ message }}</li>
      {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
```

Ici, on affiche les messages flash dans une liste.

## Macros Jinja

Pour éviter de répéter le code HTML, on peut utiliser les macros Jinja.
Nous allons ici créer une macro pour afficher les champs de saisie d'un formulaire.

```html
{% macro displayInput(field, showErrors="true") %}
  <div class="form-group">
    {{ field.label }}
    {{ field(class="form-control", **kwargs)}}
    
    {% if showErrors and field.errors %}
      <div class="alert alert-danger">
        {% for error in field.errors %}
          <span class="help-inline">{{ error }}</span>
        {% endfor %}
      </div>
    {% endif %}
  </div>
{% endmacro %}
```

Ici, on crée une macro `displayInput` qui prend deux paramètres :
- `field` : le champ de saisie à afficher
- `showErrors` : afficher les erreurs de validation (boolean)

## Utiliser une macro

Pour utiliser une macro, on utilise la balise `import`.

```html
{% import 'macros.html' as macros %}

<form action="" method="POST">
  {{ macros.displayInput(form.email) }}
  {{ macros.displayInput(form.pwd) }}
  {{ macros.displayInput(form.pwd_confirm) }}
  {{ form.submit }}
</form>
```